### Database connection

In [1]:
from langchain_community.utilities import SQLDatabase

In [2]:
db_path = '../data/inventory.db'
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print("dialect:", db.dialect)
print("tables:", db.get_usable_table_names())

dialect: sqlite
tables: ['Orders', 'Stock']


In [3]:
records = db.run("SELECT * FROM Stock LIMIT 1")
print(records)

[('1009AA', 7210.0, 'Kg', 30, 48, 28.76326)]


In [4]:
info = db.get_table_info(["Stock"])
print(info)


CREATE TABLE "Stock" (
	id TEXT, 
	current_stock_quantity FLOAT, 
	units TEXT, 
	avg_lead_time_days BIGINT, 
	maximum_lead_time_days BIGINT, 
	unit_price FLOAT
)

/*
3 rows from Stock table:
id	current_stock_quantity	units	avg_lead_time_days	maximum_lead_time_days	unit_price
1009AA	7210.0	Kg	30	48	28.76326
1077CA	46516.0	Kg	45	70	22.9777
1083AA	48210.0	Kg	45	68	29.02
*/


In [5]:
records = db.run("SELECT * FROM Orders LIMIT 1")
print(records)

[('2022-11-19 00:00:00.000000', '3084CA', 1.0)]


In [6]:
info = db.get_table_info(["Orders"])
print(info)



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/


### LLM

In [7]:
from langchain_ollama import ChatOllama


llm = ChatOllama(model="qwen2.5-coder:3b")

### Tools

In [8]:
# tools
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
toolkit

SQLDatabaseToolkit(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000255E08B0050>, llm=ChatOllama(model='qwen2.5-coder:3b'))

### SQL agent

In [9]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db,agent_type="openai-tools",verbose=True)


#### Example 1

In [10]:
#expected_query = "SELECT COUNT(*) FROM Orders"
#expected result = 33919
query = "How many orders are there in the Orders table?"
response = agent_executor.invoke(query)
print("response:", response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Stock
Invoking: `sql_db_query_checker` with `{'query': 'SELECT COUNT(*) FROM Orders'}`


The provided SQLite query is already correct and does not contain any common errors mentioned:

- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Therefore, the final SQL query remains:

```sql
SELECT COUNT(*) FROM Orders
```
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM Orders'}`


[(33919,)]Based on the query `SELECT COUNT(*) FROM Orders`, there are 33919 orders in the Orders table.

> Finished chain.
response: {'input': 'How many orders are there in the Orders table?', 'output': 'Based on the query `SELECT COUNT(*) FROM Order

Resultado: Correcto.

La latencia es de 9 minutos y 57s.

In [11]:
db.run('SELECT COUNT(*) FROM Orders;')

'[(33919,)]'

#### Example 2

In [15]:
expected_query= "SELECT * FROM Orders WHERE quantity = 475"
# Example 2
query2 = "Select all the columns in the Orders table that have a quantity equal 475 "
response2 = agent_executor.invoke(query2)
print("response:", response2)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_schema` with `{'table_names': 'Orders'}`



CREATE TABLE "Orders" (
	date DATETIME, 
	id TEXT, 
	quantity FLOAT
)

/*
3 rows from Orders table:
date	id	quantity
2022-11-19 00:00:00	3084CA	1.0
2022-06-30 00:00:00	2621AA	20.0
2023-04-19 00:00:00	1193BA	147.0
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT * FROM Orders WHERE quantity = 475 LIMIT 10'}`


The provided SQL query is syntactically correct and does not contain any common mistakes that could be addressed. Therefore, it can be reproduced as-is:

```sql
SELECT * FROM Orders WHERE quantity = 475 LIMIT 10;
```

There are no errors or potential improvements to suggest in this query.
Invoking: `sql_db_query` with `{'query': 'SELECT * FROM Orders WHERE quantity = 475 LIMIT 10'}`


[('2023-04-16 00:00:00.000000', '2785BA', 475.0), ('2022-05-23 00:00:00.000000', '1993AA', 475.0), ('2022-05-23 00:00:00.000000', '2727CA', 475.0), ('2022-04-03 00:00:00.000000', '28

In [16]:
db.run("SELECT * FROM Orders WHERE quantity = 475 LIMIT 10")

"[('2023-04-16 00:00:00.000000', '2785BA', 475.0), ('2022-05-23 00:00:00.000000', '1993AA', 475.0), ('2022-05-23 00:00:00.000000', '2727CA', 475.0), ('2022-04-03 00:00:00.000000', '2814BA', 475.0), ('2022-09-23 00:00:00.000000', '2847BA', 475.0), ('2022-08-20 00:00:00.000000', '2291AA', 475.0), ('2022-03-28 00:00:00.000000', '2803BA', 475.0)]"